In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from collections import Counter
from Method.GradientBoost import XGBoostRegressor as xgb
from Method.DecisionTree import DecisionTree

In [2]:
train_data = pd.read_csv("Processed Data/train.csv")
validation_data = pd.read_csv("Processed Data/validation.csv")

In [3]:
X_train = train_data.drop(columns=['Attrition_rate']).values
y_train = train_data['Attrition_rate'].values

In [4]:
model = xgb()
model.fit(X_train, y_train)

In [5]:
X_val = validation_data.drop(columns=['Attrition_rate']).values
y_val = validation_data['Attrition_rate'].values

y_pred = model.predict(X_val)

In [6]:
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE on validation set: {rmse}")

RMSE on validation set: 0.18099281805242315


f:\Apps\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [7]:
test_data = pd.read_csv("Processed Data/test.csv")
X_test = test_data.values

In [8]:
y_pred_test = model.predict(X_test)

In [9]:
test_data['Attrition_rate'] = y_pred_test
test_data.to_csv('test_predictions.csv', index=False)

Cross Validation